In [155]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import os
from scipy.stats import pearsonr

In [135]:
def string2datetime(stringDTSeries):
    try:
        return [(dt.datetime.strptime(DT,'%Y/%m/%d %I:%M:%S %p')) for DT in stringDTSeries]
    except(ValueError):
        return [(dt.datetime.strptime(DT,'%Y/%m/%d %H:%M:%S')) for DT in stringDTSeries]

In [163]:
path = '/Volumes/RT&O/C Brace/GPS Analysis/UIC_GPSData/'

In [164]:
os.listdir(path)

['CBR08_Final_Summary_File.xlsx',
 'CBR11_Final_Summary_File.xls',
 'CBR15_Final_Summary_File.xlsx',
 'CBR16_Final_Summary_File.xlsx',
 'CBR19_Final_Summary_File.xlsx',
 'CBR21_Final_Summary_File.xlsx',
 'CBR22_Final_Summary_File.xlsx',
 'CBR24_Final_SummaryRLU.xlsx',
 'CBR25_Final_Summary_Rname.xlsx',
 'CBR26_Final_Summary.xlsx',
 'CBR27_Final_Summary_Rname.xlsx',
 'CBR28_Final_Summary_Rname.xlsx',
 'CBR30_Final_Summary_Rnames.xlsx',
 '~$CBR08_Final_Summary_File.xlsx']

## Add LandUse coordinates from GPS Data

In [165]:
Baseline = pd.read_excel(path+'CBR08_Final_Summary_File.xlsx')

In [166]:
Baseline.LandUse.unique()

array(['trip', 'Non-Parcel Road ROW', 'home', 'INST_EDU_K12', 'COM_URBMIX',
       'COM_REGIONAL', 'RES_MF', 'COM_BIGBOX', 'COM_URBMIXwRES',
       'COM_MALL', 'INST_MEDICAL'], dtype=object)

In [167]:
Gpsdata = pd.read_csv('./GPS Data/CBR08_Baseline.csv')

In [175]:
Baseline.head(3)

,Date,Sequence,BeginTime,EndTime,Mode,LandUse,MeanSpeed,Numberpoints,TotalGap,Duration
0,2014-11-20,1,12:16:42,13:01:42,Car,trip,18.802146,269,1.226666,45.0
1,2014-11-20,2,13:01:52,13:16:22,NaN,Non-Parcel Road ROW,0.134054,88,0.293333,14.5
2,2014-11-20,3,13:16:32,13:17:32,Car,trip,28.091538,7,0.023333,1.0


In [176]:
Gpsdata.head(3)

,INDEX,TRACK ID,VALID,UTC DATE,UTC TIME,LOCAL DATE,LOCAL TIME,MS,LATITUDE,N/S,LONGITUDE,E/W,ALTITUDE,SPEED,HEADING,G-X,G-Y,G-Z
0,1,1,FIXED,2014/11/20,18:16:42,2014/11/20,13:16:42,0,41.894528,N,87.620507,W,150.075867,3.069808,172.784179,0.0,0.0,0.0
1,2,1,FIXED,2014/11/20,18:16:52,2014/11/20,13:16:52,0,41.893873,N,87.620396,W,150.352463,0.941980,195.028903,0.0,0.0,0.0
2,3,1,FIXED,2014/11/20,18:17:2,2014/11/20,13:17:2,0,41.893369,N,87.620578,W,152.352982,9.673010,2.266546,0.0,0.0,0.0


In [170]:
x = Gpsdata['LOCAL DATE']+' '+Gpsdata['LOCAL TIME'] #full date/time
GpsDates = string2datetime(x) #convert to datetime object
GpsDates=pd.Series(GpsDates)

In [122]:
Gpsdata['Full Local Date'] = Gpsdata.apply(dt.datetime.strptime)

'2014/11/20'

In [171]:
loc_lat = []
loc_lon = []

for i in range(10):
    start = Baseline.BeginTime[i]
    end = Baseline.EndTime[i]
    d = Baseline.Date[i]
    d = d.strftime('%Y-%m-%d') 
    startdate = dt.datetime.strptime(d+' '+start,'%Y-%m-%d %H:%M:%S')
    enddate = dt.datetime.strptime(d+' '+end,'%Y-%m-%d %H:%M:%S')
       
    ind = (GpsDates > startdate) & (GpsDates < enddate)
    meanLat = np.mean(Gpsdata['LATITUDE'][ind])
    meanLon = np.mean(Gpsdata['LONGITUDE'][ind])
    loc_lat.append(meanLat)
    loc_lon.append(meanLon)


In [172]:
loc_lat

[nan,
 nan,
 41.8942158,
 41.86598685617973,
 41.85632984000001,
 41.85636240506329,
 41.85641747368422,
 41.85611473379714,
 41.85644931521085,
 41.856278]

# TODO:
* Daylight savings
* Add Wear time fraction for each row
* Loop through files and generate data for each brace (need to rename xls sheets)